In [2]:
import pprint as pp
import json
import torch
from transformers import AutoTokenizer, T5ForConditionalGeneration

In [3]:
params = {
    "max_length": 256,
    "min_length": 8,
    "no_repeat_ngram_size": 3,
    "early_stopping": True,
    "repetition_penalty": 3.5,
    "length_penalty": 0.3,
    "encoder_no_repeat_ngram_size": 3,
    "num_beams": 4,
} # parameters for text generation out of model


In [5]:
import requests 
from bs4 import BeautifulSoup 
import re

URL_list = ['https://edition.cnn.com/2023/04/05/politics/donald-trump-court-2024/index.html', 
            'https://edition.cnn.com/travel/article/worlds-busiest-airports-2022-aci/index.html',
            'https://edition.cnn.com/2023/04/04/business/elon-musk-forbes-billionaire-list/index.html',
            'https://edition.cnn.com/2023/04/04/business/australia-aesop-loreal-natura-deal-intl-hnk/index.html',
            'https://edition.cnn.com/2023/04/03/investing/china-renaissance-delays-results-missing-ceo-intl-hnk/index.html'
            ]

headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36'}

news_data = []
cnt=1
for url in URL_list:
    news = []
    data = requests.get(url, headers=headers)
    soup = BeautifulSoup(data.text, 'html.parser')
    title = soup.select_one('#maincontent')
    title_text=title.text.strip()
   
    content = soup.select_one('body > div.layout__content-wrapper.layout-with-rail__content-wrapper > section.layout__wrapper.layout-with-rail__wrapper > section.layout__main-wrapper.layout-with-rail__main-wrapper > section.layout__main.layout-with-rail__main > article > section > main > div.article__content-container > div.article__content')
    context_text = re.sub(r'\s+',' ', content.text)

    news.append(title_text)
    news.append(context_text)
    news_data.append(news)
    print(cnt)
    cnt += 1

print(news_data[0][0]) #제목
print(news_data[0][1]) #내용

1
2
3
4
5
A ‘surreal’ day for Trump in court may only tear the country further apart
 CNN — There were two big things to fear before Tuesday’s momentous first criminal arraignment of an ex-president of the United States. The first was that while Manhattan District Attorney Alvin Bragg’s case might cite solid evidence of alleged wrongdoing by Donald Trump, his legal theory might prove too vague, complex and seem like too much of a throwback to a 7-year-old election to be an easy sell to the public. The other was that Trump would respond with such fury and incitement that he would rip even deeper divides in a nation estranged by his aberrant presidency and stoke new turmoil that could further damage vital political and judicial institutions. More on the historic indictment of Donald Trump Trump was indicted related to a hush money payment scheme probe. Here’s what we knowTakeaways and key lines from the indictmentREAD the full indictment and statement of factsFact check: Trump delivers b

In [7]:
title = []
text = []

for i in range(5):
  title.append(news_data[i][0])
  
for i in range(5):
  text.append(news_data[i][1])

In [8]:
model_name = "pszemraj/long-t5-tglobal-base-16384-book-summary"
model = T5ForConditionalGeneration.from_pretrained(model_name)
sum_tokenizer = AutoTokenizer.from_pretrained('t5-base')

c:\Users\frmis\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:129: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\frmis\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
You are using a model of type longt5 to instantiate a model of type t5. This is not supported for all configurations of

Some weights of the model checkpoint at pszemraj/long-t5-tglobal-base-16384-book-summary were not used when initializing T5ForConditionalGeneration: ['encoder.block.3.layer.0.TransientGlobalSelfAttention.q.weight', 'encoder.block.3.layer.0.TransientGlobalSelfAttention.o.weight', 'encoder.block.9.layer.0.TransientGlobalSelfAttention.v.weight', 'encoder.block.2.layer.0.TransientGlobalSelfAttention.v.weight', 'encoder.block.1.layer.0.TransientGlobalSelfAttention.k.weight', 'encoder.block.4.layer.0.TransientGlobalSelfAttention.v.weight', 'encoder.block.0.layer.0.TransientGlobalSelfAttention.relative_attention_bias.weight', 'encoder.block.0.layer.0.TransientGlobalSelfAttention.k.weight', 'encoder.block.1.layer.0.TransientGlobalSelfAttention.o.weight', 'encoder.block.9.layer.0.TransientGlobalSelfAttention.o.weight', 'encoder.block.5.layer.0.TransientGlobalSelfAttention.v.weight', 'encoder.block.1.layer.0.TransientGlobalSelfAttention.q.weight', 'encoder.block.0.layer.0.TransientGlobalSelfAtte

c:\Users\frmis\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\models\t5\tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [14]:
device =  'cpu'

In [15]:
def generate_summary(text):
    input_ids = sum_tokenizer.encode(text, return_tensors='pt', max_length=512, truncation=True).to(device)
    summary_ids = model.generate(input_ids.to(device), max_length=150, num_beams=2, early_stopping=True)
    summary = sum_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [16]:
model.to('cpu')

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [17]:
for i in range(len(text)):
    summary = generate_summary(text[i])
    print(title[i])
    print(summary)

A ‘surreal’ day for Trump in court may only tear the country further apart
The Trumpet is a major figure in the American political scene, as well as many other nations. It has been used to symbolize power and influence-especially in Britain since the end of the Civil War.
